### Vector stores and retrievals
This video tutorial will familiarize you with LangChain's vector store retrievar abstractions. These abstractions are designed to support retrieval of data-- from(vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inferences, as in the case of retrieval-augmented generation.

We will cover
- Documents 
- Vector stores
- Retrievals

### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

page_content: a string representing the contents;
metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

In [26]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Dogs are great companions, known for their loyalty and friendliness.",
             metadata={"source":"mammal-pets-doc"},),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadate={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source":"fish-pets-doc"},
    ),
    Document(
        page_content="Parratos are intelligent birds capable of mimicking human speech.",
        metadata={"source":"bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata= {"source":"mammal-pets-doc"},
    ),
]

In [27]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parratos are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

os.environ['HF_TOKEN']= os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key= groq_api_key,model='llama-3.1-8b-instant')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022E92808F50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022E9280B800>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [29]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [30]:
## Vector stores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embedding= embeddings) 
vectorstore

In [31]:
vectorstore.similarity_search('cat')

[Document(id='f86aa27f-532a-44d4-83e6-1b0e1d2dd091', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='e86720e7-0356-4aab-99dd-64fae5958f51', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='69937059-53a3-498a-81af-b1278fa559f0', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='916486c4-581d-419f-baa3-eeca1f89f7a8', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [32]:
## Async query
await vectorstore.asimilarity_search('cat')

[Document(id='f86aa27f-532a-44d4-83e6-1b0e1d2dd091', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='e86720e7-0356-4aab-99dd-64fae5958f51', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='69937059-53a3-498a-81af-b1278fa559f0', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='916486c4-581d-419f-baa3-eeca1f89f7a8', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [33]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='f86aa27f-532a-44d4-83e6-1b0e1d2dd091', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='e86720e7-0356-4aab-99dd-64fae5958f51', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='69937059-53a3-498a-81af-b1278fa559f0', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='916486c4-581d-419f-baa3-eeca1f89f7a8', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956)]

## Retrievers

LangChain VectorStore object do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g. synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [34]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='f86aa27f-532a-44d4-83e6-1b0e1d2dd091', metadata={}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='e4948a0e-312b-47ad-951f-cce75fd7799c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

VectorStores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retreivers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [35]:
retriever =vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
) 

retriever.batch(["cat","dog"])

[[Document(id='f86aa27f-532a-44d4-83e6-1b0e1d2dd091', metadata={}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='e4948a0e-312b-47ad-951f-cce75fd7799c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain ={"context":retriever, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about dogs")

print(response.content)

Dogs are great companions, known for their loyalty and friendliness.
